## Final

In [134]:
import re
import os
import random
import math
import json
import time
import pandas as pd
import numpy as np
import copy as copy
import matplotlib.pyplot as plt
import translators as ts
import translators.server as tss
from translate import Translator
from pandas import Timestamp
from datetime import datetime
from os import getcwd
from os.path import join
%matplotlib inline


In [146]:
# Load World Mortality Data
path = join(getcwd().rstrip('src'),
            'data/world_mortality.csv').replace('\\', '/')
DF = pd.read_csv(path)
DF.rename(columns={'country_name': 'country'}, inplace=True)

# Separate Data by Year
DF_Regular = DF[DF.year < 2020]
DF_2020 = DF[(DF.year == 2020)]
DF_2021 = DF[(DF.year == 2021)]
DF_2022 = DF[(DF.year == 2022)]

# Data for 2015-2019
Regular = DF_Regular.groupby('country').mean().reset_index()
Regular['AverageAnnualUnitCount'] = round(Regular.time * 2 - 1, 0)
tmp = copy.copy(DF)
time_unit_dict = {'monthly': 12, 'weekly': 52}
tmp['AnnualUnitCount'] = [time_unit_dict[x] for x in tmp.time_unit]
tmp = tmp.groupby('country').mean().reset_index()[
    ['country', 'AnnualUnitCount']]
Regular = pd.merge(Regular, tmp, on='country')
Regular['deaths'] = Regular.deaths * Regular.AnnualUnitCount
Regular = Regular[['country', 'deaths', 'AverageAnnualUnitCount']]

# Data for 2020
DF_2020 = DF_2020.groupby('country').mean().reset_index()
DF_2020['AverageAnnualUnitCount'] = round(DF_2020.time * 2 - 1, 0)
DF_2020 = pd.merge(DF_2020, tmp, on='country')
DF_2020['deaths'] = DF_2020.deaths * DF_2020.AnnualUnitCount
DF_2020 = DF_2020[['country', 'deaths', 'AverageAnnualUnitCount']]

# Data for 2021
DF_2021 = DF_2021.groupby('country').mean().reset_index()
DF_2021['AverageAnnualUnitCount'] = round(DF_2021.time * 2 - 1, 0)
DF_2021 = pd.merge(DF_2021, tmp, on='country')
DF_2021['deaths'] = DF_2021.deaths * DF_2021.AnnualUnitCount
DF_2021 = DF_2021[['country', 'deaths', 'AverageAnnualUnitCount']]

# Data for 2022
DF_2022 = DF_2022.groupby('country').mean().reset_index()
DF_2022['AverageAnnualUnitCount'] = round(DF_2022.time * 2 - 1, 0)
DF_2022 = pd.merge(DF_2022, tmp, on='country')
DF_2022['deaths'] = DF_2022.deaths * DF_2022.AnnualUnitCount
DF_2022 = DF_2022[['country', 'deaths', 'AverageAnnualUnitCount']]

# Merge Data (2015-2019, 2020, 2021, 2022)
DF_2020.rename(columns={'deaths': '2020Deaths'}, inplace=True)
DF_2021.rename(columns={'deaths': '2021Deaths'}, inplace=True)
DF_2022.rename(columns={'deaths': '2022Deaths'}, inplace=True)
newDF = pd.merge(Regular, DF_2020[['country', '2020Deaths']], on='country')
newDF = pd.merge(newDF, DF_2021[['country', '2021Deaths']], on='country')
newDF = pd.merge(newDF, DF_2022[['country', '2022Deaths']], on='country')

# Excess Deaths (2020, 2021, 2022)
newDF['Excess2020'] = newDF['2020Deaths'] - newDF['deaths']
newDF['Excess2021'] = newDF['2021Deaths'] - newDF['deaths']
newDF['Excess2022'] = newDF['2022Deaths'] - newDF['deaths']

# Merge with OWID Population, Continent Data
path = join(getcwd().rstrip('src'),
            'data/owid-covid-data.csv').replace('\\', '/')
data = pd.read_csv(path)
data = data[['continent', 'location', 'population']]
data.rename(columns={'location': 'country'}, inplace=True)
data.groupby('country').count().sort_values(by='population', ascending=False)
data.drop_duplicates(subset=['country'], inplace=True)
data.reset_index(drop=True, inplace=True)
newDF = pd.merge(newDF, data, on='country')


# Death Rate (Death by Population)
DR_2022 = newDF['2022Deaths'] / newDF['population']
DR_2021 = newDF['2021Deaths'] / newDF['population']
DR_2020 = newDF['2020Deaths'] / newDF['population']
DR_2022.rename('DR_2022', inplace=True)
DR_2021.rename('DR_2021', inplace=True)
DR_2020.rename('DR_2020', inplace=True)
newDF = pd.concat([newDF, DR_2022, DR_2021, DR_2020], axis=1)

# Excess Deaths Rate (Excess Deaths by Population)
EDR_2022 = newDF['Excess2022'] / newDF['population']
EDR_2021 = newDF['Excess2021'] / newDF['population']
EDR_2020 = newDF['Excess2020'] / newDF['population']
EDR_2022.rename('EDR_2022', inplace=True)
EDR_2021.rename('EDR_2021', inplace=True)
EDR_2020.rename('EDR_2020', inplace=True)
newDF = pd.concat([newDF, EDR_2022, EDR_2021, EDR_2020], axis=1)

# newDF.head()


C:\Users\dachu\AppData\Local\Temp\ipykernel_21028\4150400861.py:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  Regular = DF_Regular.groupby('country').mean().reset_index()
C:\Users\dachu\AppData\Local\Temp\ipykernel_21028\4150400861.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tmp = tmp.groupby('country').mean().reset_index()[
C:\Users\dachu\AppData\Local\Temp\ipykernel_21028\4150400861.py:26: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which sho

['Europe', 'Asia', 'Oceania', 'North America', 'South America', 'Africa']


C:\Users\dachu\AppData\Local\Temp\ipykernel_21028\4150400861.py:99: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  Continent = newDF.groupby('continent').sum().reset_index()


,continent,deaths,AverageAnnualUnitCount,2020Deaths,2021Deaths,2022Deaths,Excess2020,Excess2021,Excess2022,population,DR_2022,DR_2021,DR_2020,EDR_2022,EDR_2021,EDR_2020
0,Africa,1.100468e+06,76.0,1.261603e+06,1.451975e+06,1.198143e+06,1.611346e+05,3.515070e+05,97675.255814,169953348.0,0.025200,0.028817,0.024451,0.003151,0.006768,0.002402
1,Asia,3.990309e+06,360.0,4.337018e+06,4.882768e+06,4.669033e+06,3.467085e+05,8.924584e+05,678723.227586,617431194.0,0.132378,0.140179,0.127847,0.017613,0.025414,0.013082
2,Europe,7.959424e+06,1744.0,8.940950e+06,9.468145e+06,8.655812e+06,9.815252e+05,1.508721e+06,696387.311468,735384061.0,0.465297,0.505783,0.472801,0.041313,0.081799,0.048817
3,North America,3.875724e+06,296.0,4.880254e+06,5.011093e+06,4.556194e+06,1.004531e+06,1.135369e+06,680470.478544,523107334.0,0.072416,0.073058,0.068457,0.011379,0.012021,0.007421
4,Oceania,1.930551e+05,104.0,1.941631e+05,2.057520e+05,2.296409e+05,1.108014e+03,1.269692e+04,36585.833826,31050817.0,0.014943,0.013378,0.012583,0.002386,0.000821,0.000026


In [162]:
# Load World Mortality Data
path = join(getcwd().rstrip('src'),
            'data/world_mortality.csv').replace('\\', '/')
DF1 = pd.read_csv(path)
DF1.rename(columns={'country_name': 'country'}, inplace=True)

# Deaths by Country
# DF1 = DF1.groupby('country').sum().sort_values(by='deaths', ascending=False)
# DF1.drop(columns=['year', 'time'], inplace=True)
# DF1.head()
DF1 = DF1.groupby('country').mean().reset_index()
DF1['AverageAnnualUnitCount'] = round(DF1.time * 2 - 1, 0)
tmp = copy.copy(DF)
time_unit_dict = {'monthly': 12, 'weekly': 52}
tmp['AnnualUnitCount'] = [time_unit_dict[x] for x in tmp.time_unit]
tmp = tmp.groupby('country').mean().reset_index()[
    ['country', 'AnnualUnitCount']]
DF1 = pd.merge(DF1, tmp, on='country')
DF1['deaths'] = DF1.deaths * DF1.AnnualUnitCount
DF1 = DF1[['country', 'deaths', 'AverageAnnualUnitCount']]

# Merge with OWID Population, Continent Data
path = join(getcwd().rstrip('src'),
            'data/owid-covid-data.csv').replace('\\', '/')
data = pd.read_csv(path)
data = data[['continent', 'location', 'population']]
data.rename(columns={'location': 'country'}, inplace=True)
data.groupby('country').count().sort_values(by='population', ascending=False)
data.drop_duplicates(subset=['country'], inplace=True)
data.reset_index(drop=True, inplace=True)
DF1 = pd.merge(DF1, data, on='country')

# Separate Data by Continent
print(DF1.continent.unique().tolist())
Africa = DF1[DF1.continent == 'Africa']
Asia = DF1[DF1.continent == 'Asia']
Europe = DF1[DF1.continent == 'Europe']
NorthAmerica = DF1[DF1.continent == 'North America']
Oceania = DF1[DF1.continent == 'Oceania']
SouthAmerica = DF1[DF1.continent == 'South America']

# Create Continent Data
Continent = DF1.groupby('continent').sum().reset_index()

# Death Rate (Death by Population)
DR = Continent['deaths'] / Continent['population']
DR.rename('DR', inplace=True)
Continent = pd.concat([Continent, DR], axis=1)

# # Asia.head()
Continent.head()
# DF1.head()


C:\Users\dachu\AppData\Local\Temp\ipykernel_21028\2814141361.py:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  DF1 = DF1.groupby('country').mean().reset_index()
C:\Users\dachu\AppData\Local\Temp\ipykernel_21028\2814141361.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tmp = tmp.groupby('country').mean().reset_index()[


['Europe', 'Africa', 'North America', 'South America', 'Asia', 'Oceania']


C:\Users\dachu\AppData\Local\Temp\ipykernel_21028\2814141361.py:44: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  Continent = DF1.groupby('continent').sum().reset_index()


,continent,deaths,AverageAnnualUnitCount,population,DR
0,Africa,1.446724e+06,111.0,226500733.0,0.006387
1,Asia,4.328424e+06,441.0,654272504.0,0.006616
2,Europe,8.468331e+06,1748.0,745073933.0,0.011366
3,North America,4.518448e+06,447.0,572155415.0,0.007897
4,Oceania,2.010631e+05,126.0,31642649.0,0.006354


In [213]:
path = join(getcwd().rstrip('src'),
            'data/owid-covid-data.csv').replace('\\', '/')
print(path)
DF2 = pd.read_csv(path)
print(DF2.columns.tolist())
DF2 = DF2[['continent', 'location', 'date', 'population', 'total_cases',
           'new_vaccinations', 'total_vaccinations_per_hundred']]
DF2.head()


d:/Note_Database/Subject/BD_ML Big Data and Machine Learning/BD_ML_Code/data/owid-covid-data.csv
['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths', 'new_deaths', 'new_deaths_smoothed', 'total_cases_per_million', 'new_cases_per_million', 'new_cases_smoothed_per_million', 'total_deaths_per_million', 'new_deaths_per_million', 'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients', 'icu_patients_per_million', 'hosp_patients', 'hosp_patients_per_million', 'weekly_icu_admissions', 'weekly_icu_admissions_per_million', 'weekly_hosp_admissions', 'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests', 'total_tests_per_thousand', 'new_tests_per_thousand', 'new_tests_smoothed', 'new_tests_smoothed_per_thousand', 'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'new_vaccinations', 'new_vaccinations_smoothed', 'total_vaccinati

,continent,location,date,population,total_cases,new_vaccinations,total_vaccinations_per_hundred
0,Asia,Afghanistan,2020-02-24,40099462.0,5.0,NaN,NaN
1,Asia,Afghanistan,2020-02-25,40099462.0,5.0,NaN,NaN
2,Asia,Afghanistan,2020-02-26,40099462.0,5.0,NaN,NaN
3,Asia,Afghanistan,2020-02-27,40099462.0,5.0,NaN,NaN
4,Asia,Afghanistan,2020-02-28,40099462.0,5.0,NaN,NaN


## Q1 Which in the following list is most vulnerable to COVID-19 death?  Please note that a good medical system implies adaquate vaccination is available.


1. Baby in a good medical system.
2. Age 40 and sick in a bad medical system
3. Age 50 in a bad medical system.
4. Age 60 and sick in a good medical system
5. Age 70 without vaccination in a good medical system.


### ANS: 1?

## Q2 Which continent has the lowest deaths per million poeple from COVID-19

1. Europe
2. Asia
3. North America
4. South America
5. Africa
6. Oceania


In [163]:
Continent.sort_values(by='DR', ascending=True, inplace=True)
Continent.reset_index(drop=True, inplace=True)
Continent.head()


,continent,deaths,AverageAnnualUnitCount,population,DR
0,Oceania,2.010631e+05,126.0,31642649.0,0.006354
1,South America,2.573910e+06,276.0,404948473.0,0.006356
2,Africa,1.446724e+06,111.0,226500733.0,0.006387
3,Asia,4.328424e+06,441.0,654272504.0,0.006616
4,North America,4.518448e+06,447.0,572155415.0,0.007897


### ANS: Oceania

## Q3 Which country is the 2nd country that reaches 100 total vaccination per 100 people?

1. Spain
2. Israel
3. United Kingdom
4. Norway
5. New Zealand


In [215]:
options = ['Spain', 'Israel', 'United Kingdom', 'Norway', 'New Zealand']
DF2_tmp = DF2[DF2.location.isin(options)]

DF2_tmp = DF2_tmp[DF2_tmp.total_vaccinations_per_hundred >= 100]
DF2_tmp = DF2_tmp.groupby('location').min().reset_index()
DF2_tmp = DF2_tmp.sort_values(by='date', ascending=True)

DF2_tmp.head()


,location,continent,date,population,total_cases,new_vaccinations,total_vaccinations_per_hundred
0,Israel,Asia,2021-03-14,9291000.0,818977.0,3.0,100.48
4,United Kingdom,Europe,2021-06-04,67281040.0,4506018.0,3103.0,100.05
3,Spain,Europe,2021-07-11,47486935.0,3937192.0,22084.0,100.61
2,Norway,Europe,2021-07-29,5403021.0,137127.0,1.0,100.39
1,New Zealand,Oceania,2021-09-28,5129728.0,4248.0,109.0,100.20


### ANS: United Kingdom

## Q4 Other than China, which country reached 5000 total confirmed cases first?

1. Japan
2. Korea
3. Italy
4. Iran
5. Russia


In [212]:
# print(DF2.location.unique().tolist())

options = ['Japan', 'South Korea', 'North Korea', 'Italy', 'Iran', 'Russia']
DF2_tmp = DF2[DF2.location.isin(options)]

DF2_tmp = DF2_tmp[DF2_tmp.total_cases >= 5000]
DF2_tmp = DF2_tmp.groupby('location').min().reset_index()

DF2_tmp.sort_values(by='date', ascending=True, inplace=True)

DF2_tmp.head(6)

,location,continent,date,population,total_cases,new_vaccinations
4,South Korea,Asia,2020-03-03,51830139.0,5186.0,32.0
0,Iran,Asia,2020-03-07,87923432.0,5823.0,3000.0
1,Italy,Europe,2020-03-07,59240330.0,5883.0,196.0
3,Russia,Europe,2020-04-05,145102755.0,5389.0,215.0
2,Japan,Asia,2020-04-08,124612530.0,5020.0,339.0


### ANS: South Korea ??

## Q5 Which of the following country is not reported in world_mortality.csv?

1. United States
2. China
3. Taiwan
4. Macau
5. Malta


In [136]:
countries = DF.country.unique().tolist()
print(countries)
print()
options = ['United States', 'China', 'Taiwan', 'Macau', 'Malta']
for option in options:
    if option in countries:
        print('{} is in the list'.format(option))
    else:
        print('{} is not in the list'.format(option))
DF.head()

['Albania', 'Algeria', 'Andorra', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Bermuda', 'Bolivia', 'Bosnia', 'Brazil', 'Brunei', 'Bulgaria', 'Cabo Verde', 'Canada', 'Chile', 'Colombia', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Estonia', 'Faroe Islands', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'Georgia', 'Germany', 'Gibraltar', 'Greece', 'Greenland', 'Guadeloupe', 'Guatemala', 'Hong Kong', 'Hungary', 'Iceland', 'Iran', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kosovo', 'Kuwait', 'Kyrgyzstan', 'Latvia', 'Lebanon', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao', 'Malaysia', 'Maldives', 'Malta', 'Martinique', 'Mauritius', 'Mayotte', 'Mexico', 'Moldova', 'Monaco', 'Mongolia', 'Montenegro', 'Netherlands', 'New Caledonia', 'New Zealand', 'Nicaragua', 'N

,iso3c,country,year,time,time_unit,deaths
0,ALB,Albania,2015,1,monthly,2490.0
1,ALB,Albania,2015,2,monthly,2139.0
2,ALB,Albania,2015,3,monthly,2051.0
3,ALB,Albania,2015,4,monthly,1906.0
4,ALB,Albania,2015,5,monthly,1709.0


### ANS: China and Macau

## Q6 Which country has the highest deaths in 2020 as reported in world_mortality.csv?

1. United States
2. China
3. Russia
4. Italy
5. South Africa


In [137]:
# Separate Data by Year
DF_Regular = DF[DF.year < 2020]
DF_2020 = DF[(DF.year == 2020)]
DF_2021 = DF[(DF.year == 2021)]
DF_2022 = DF[(DF.year == 2022)]

# Data for 2015-2019
Regular = DF_Regular.groupby('country').mean().reset_index()
Regular['AverageAnnualUnitCount'] = round(Regular.time * 2 - 1, 0)
tmp = copy.copy(DF)
time_unit_dict = {'monthly': 12, 'weekly': 52}
tmp['AnnualUnitCount'] = [time_unit_dict[x] for x in tmp.time_unit]
tmp = tmp.groupby('country').mean().reset_index()[
    ['country', 'AnnualUnitCount']]
Regular = pd.merge(Regular, tmp, on='country')
Regular['deaths'] = Regular.deaths * Regular.AnnualUnitCount
Regular = Regular[['country', 'deaths', 'AverageAnnualUnitCount']]

# Data for 2020
DF_2020 = DF_2020.groupby('country').mean().reset_index()
DF_2020['AverageAnnualUnitCount'] = round(DF_2020.time * 2 - 1, 0)
DF_2020 = pd.merge(DF_2020, tmp, on='country')
DF_2020['deaths'] = DF_2020.deaths * DF_2020.AnnualUnitCount
DF_2020 = DF_2020[['country', 'deaths', 'AverageAnnualUnitCount']]

DF_2020.head()
DF_2020.sort_values(by='deaths', ascending=False).head(10)

C:\Users\dachu\AppData\Local\Temp\ipykernel_21028\2497503443.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  Regular = DF_Regular.groupby('country').mean().reset_index()
C:\Users\dachu\AppData\Local\Temp\ipykernel_21028\2497503443.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tmp = tmp.groupby('country').mean().reset_index()[
C:\Users\dachu\AppData\Local\Temp\ipykernel_21028\2497503443.py:20: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which shou

,country,deaths,AverageAnnualUnitCount
119,United States,3.375903e+06,53.0
96,Russia,2.138586e+06,12.0
18,Brazil,1.556824e+06,12.0
56,Japan,1.384544e+06,12.0
74,Mexico,1.066931e+06,53.0
42,Germany,9.825528e+05,53.0
54,Italy,7.425816e+05,53.0
118,United Kingdom,6.835586e+05,53.0
33,Egypt,6.662000e+05,12.0
38,France,6.518004e+05,53.0


### ANS: United States

## Q7 Which country has the highest deaths per population (death rate) in 2020 as reported in world_mortality.csv?

1. Peru
2. Bulgaria
3. Russia
4. Hungary
5. Estonia


In [138]:
# # Data for 2021
# DF_2021 = DF_2021.groupby('country').mean().reset_index()
# DF_2021['AverageAnnualUnitCount'] = round(DF_2021.time * 2 - 1, 0)
# DF_2021 = pd.merge(DF_2021, tmp, on='country')
# DF_2021['deaths'] = DF_2021.deaths * DF_2021.AnnualUnitCount
# DF_2021 = DF_2021[['country', 'deaths', 'AverageAnnualUnitCount']]

# # Data for 2022
# DF_2022 = DF_2022.groupby('country').mean().reset_index()
# DF_2022['AverageAnnualUnitCount'] = round(DF_2022.time * 2 - 1, 0)
# DF_2022 = pd.merge(DF_2022, tmp, on='country')
# DF_2022['deaths'] = DF_2022.deaths * DF_2022.AnnualUnitCount
# DF_2022 = DF_2022[['country', 'deaths', 'AverageAnnualUnitCount']]

# # Merge Data (2015-2019, 2020, 2021, 2022)
# DF_2020.rename(columns={'deaths': '2020Deaths'}, inplace=True)
# DF_2021.rename(columns={'deaths': '2021Deaths'}, inplace=True)
# DF_2022.rename(columns={'deaths': '2022Deaths'}, inplace=True)
# newDF = pd.merge(Regular, DF_2020[['country', '2020Deaths']], on='country')
# newDF = pd.merge(newDF, DF_2021[['country', '2021Deaths']], on='country')
# newDF = pd.merge(newDF, DF_2022[['country', '2022Deaths']], on='country')

# # Excess Deaths (2020, 2021, 2022)
# newDF['Excess2020'] = newDF['2020Deaths'] - newDF['deaths']
# newDF['Excess2021'] = newDF['2021Deaths'] - newDF['deaths']
# newDF['Excess2022'] = newDF['2022Deaths'] - newDF['deaths']

# # Merge with OWID Population Data
# path = join(getcwd().rstrip('src'),
#             'data/owid-covid-data.csv').replace('\\', '/')
# data = pd.read_csv(path)
# data = data[['location', 'population']]
# data.rename(columns={'location': 'country'}, inplace=True)
# data.groupby('country').count().sort_values(by='population', ascending=False)
# data.drop_duplicates(subset=['country'], inplace=True)
# data.reset_index(drop=True, inplace=True)
# data.head()
# newDF = pd.merge(newDF, data, on='country')

# # Death Rate (Death by Population)
# DR_2022 = newDF['2022Deaths'] / newDF['population']
# DR_2021 = newDF['2021Deaths'] / newDF['population']
# DR_2020 = newDF['2020Deaths'] / newDF['population']
# DR_2022.rename('DR_2022', inplace=True)
# DR_2021.rename('DR_2021', inplace=True)
# DR_2020.rename('DR_2020', inplace=True)
# newDF = pd.concat([newDF, DR_2022, DR_2021, DR_2020], axis=1)
newDF.sort_values(by='DR_2020', ascending=False).head(10)

# Show Data
# newDF.head()


,country,deaths,AverageAnnualUnitCount,2020Deaths,2021Deaths,2022Deaths,Excess2020,Excess2021,Excess2022,population,DR_2022,DR_2021,DR_2020,EDR_2022,EDR_2021,EDR_2020
9,Bulgaria,1.084806e+05,52.0,1.235383e+05,148640.0,1.200707e+05,15057.697101,40159.432950,11590.099617,6885868.0,0.017437,0.021586,0.017941,0.001683,0.005832,0.002187
67,Serbia,1.022694e+05,12.0,1.168500e+05,135901.0,1.143227e+05,14580.600000,33631.600000,12053.266667,6871547.0,0.016637,0.019777,0.017005,0.001754,0.004894,0.002122
40,Lithuania,4.004936e+04,52.0,4.348574e+04,47463.0,4.210667e+04,3436.371864,7413.636015,2057.302682,2786651.0,0.015110,0.017032,0.015605,0.000738,0.002660,0.001233
38,Latvia,2.838064e+04,52.0,2.876287e+04,34471.0,2.992600e+04,382.224246,6090.356322,1545.356322,1873919.0,0.015970,0.018395,0.015349,0.000825,0.003250,0.000204
63,Romania,2.604766e+05,52.0,2.957397e+05,333028.0,2.746229e+05,35263.131063,72551.432950,14146.380319,19328560.0,0.014208,0.017230,0.015301,0.000732,0.003754,0.001824
64,Russia,1.850580e+06,12.0,2.138586e+06,2445509.0,1.931068e+06,288006.260000,594929.360000,80488.360000,145102755.0,0.013308,0.016854,0.014738,0.000555,0.004100,0.001985
28,Hungary,1.297916e+05,52.0,1.405334e+05,155032.0,1.343227e+05,10741.832430,25240.398467,4531.065134,9709786.0,0.013834,0.015967,0.014473,0.000467,0.002599,0.001106
48,Monaco,5.278000e+02,12.0,5.210000e+02,593.0,5.120000e+02,-6.800000,65.200000,-15.800000,36686.0,0.013956,0.016164,0.014202,-0.000431,0.001777,-0.000185
78,Ukraine,5.842656e+05,12.0,6.168350e+05,714263.0,6.869760e+05,32569.400000,129997.400000,102710.400000,43531422.0,0.015781,0.016408,0.014170,0.002359,0.002986,0.000748
13,Croatia,5.256503e+04,52.0,5.684777e+04,62642.0,5.733600e+04,4282.746765,10076.973180,4770.973180,4060135.0,0.014122,0.015429,0.014001,0.001175,0.002482,0.001055


### ANS: Bulgaria

## Q8 Which country has the highest excess deaths per population (death rate) in 2020 based on world_mortality.csv?
 
1. Peru
2. Bulgaria
3. Russia
4. Hungary
5. Estonia


In [139]:
# # Excess Deaths Rate (Excess Deaths by Population)
# EDR_2022 = newDF['Excess2022'] / newDF['population']
# EDR_2021 = newDF['Excess2021'] / newDF['population']
# EDR_2020 = newDF['Excess2020'] / newDF['population']
# EDR_2022.rename('EDR_2022', inplace=True)
# EDR_2021.rename('EDR_2021', inplace=True)
# EDR_2020.rename('EDR_2020', inplace=True)
# newDF = pd.concat([newDF, EDR_2022, EDR_2021, EDR_2020], axis=1)
newDF.sort_values(by='EDR_2020', ascending=False, inplace=True)
newDF.reset_index(drop=False, inplace=True)

options = ['Peru', 'Bulgaria', 'Russia', 'Hungary', 'Estonia']
for option in options:
    index = newDF[newDF.country == option].index[0]
    print('{} {}'.format(index, option))

newDF.head(10)

3 Peru
7 Bulgaria
10 Russia
30 Hungary
64 Estonia


,index,country,deaths,AverageAnnualUnitCount,2020Deaths,2021Deaths,2022Deaths,Excess2020,Excess2021,Excess2022,population,DR_2022,DR_2021,DR_2020,EDR_2022,EDR_2021,EDR_2020
0,1,Armenia,27047.400000,12.0,3.537100e+04,34638.0,27462.666667,8323.600000,7590.600000,415.266667,2790974.0,0.009840,0.012411,0.012673,0.000149,0.002720,0.002982
1,46,Mexico,696709.348659,52.0,1.066931e+06,1089683.0,836726.222222,370221.745681,392973.651341,140016.873563,126705138.0,0.006604,0.008600,0.008421,0.001105,0.003101,0.002922
2,66,San Marino,252.200000,12.0,3.430000e+02,312.0,265.333333,90.800000,59.800000,13.133333,33746.0,0.007863,0.009246,0.010164,0.000389,0.001772,0.002691
3,57,Peru,152683.900000,52.0,2.387468e+05,263448.6,175626.230000,86062.915094,110764.700000,22942.330000,33715472.0,0.005209,0.007814,0.007081,0.000680,0.003285,0.002553
4,17,Ecuador,69773.440613,52.0,1.151957e+05,104860.0,90567.100000,45422.257500,35086.559387,20793.659387,17797737.0,0.005089,0.005892,0.006472,0.001168,0.001971,0.002552
5,53,North Macedonia,20267.800000,12.0,2.540800e+04,28516.0,23560.500000,5140.200000,8248.200000,3292.700000,2103330.0,0.011202,0.013558,0.012080,0.001565,0.003921,0.002444
6,36,Kosovo,8769.200000,12.0,1.298700e+04,13019.0,9192.000000,4217.800000,4249.800000,422.800000,1782115.0,0.005158,0.007305,0.007287,0.000237,0.002385,0.002367
7,9,Bulgaria,108480.567050,52.0,1.235383e+05,148640.0,120070.666667,15057.697101,40159.432950,11590.099617,6885868.0,0.017437,0.021586,0.017941,0.001683,0.005832,0.002187
8,67,Serbia,102269.400000,12.0,1.168500e+05,135901.0,114322.666667,14580.600000,33631.600000,12053.266667,6871547.0,0.016637,0.019777,0.017005,0.001754,0.004894,0.002122
9,7,Bolivia,50612.800000,12.0,7.486300e+04,84301.0,117348.000000,24250.200000,33688.200000,66735.200000,12079472.0,0.009715,0.006979,0.006198,0.005525,0.002789,0.002008


### ANS: Peru